# 第7章: データベース

artist.json.gzは，オープンな音楽データベースMusicBrainzの中で，アーティストに関するものをJSON形式に変換し，gzip形式で圧縮したファイルである．このファイルには，1アーティストに関する情報が1行にJSON形式で格納されている．JSON形式の概要は以下の通りである．

### 60. KVSの構築

Key-Value-Store (KVS) を用い，アーティスト名（name）から活動場所（area）を検索するためのデータベースを構築せよ．

In [1]:
import gzip
import json

artists = []
with gzip.open('./data/chapter07/artist.json.gz', 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        artists.append(json.loads(line))
        
artists[:10]

[{'name': 'WIK▲N',
  'tags': [{'count': 1, 'value': 'sillyname'}],
  'sort_name': 'WIK▲N',
  'ended': True,
  'gid': '8972b1c1-6482-4750-b51f-596d2edea8b1',
  'id': 805192},
 {'name': 'Gustav Ruppke',
  'sort_name': 'Gustav Ruppke',
  'ended': True,
  'gid': 'b4f76788-7e6f-41b7-ac7b-dfb67f66282e',
  'type': 'Person',
  'id': 578352},
 {'name': 'Pete Moutso',
  'sort_name': 'Moutso, Pete',
  'ended': True,
  'gid': '49add228-eac5-4de8-836c-d75cde7369c3',
  'type': 'Person',
  'id': 371203},
 {'ended': True,
  'gid': 'c112a400-af49-4665-8bba-741531d962a1',
  'sort_name': 'Zachary',
  'id': 273232,
  'name': 'Zachary'},
 {'name': 'The High Level Ranters',
  'sort_name': 'High Level Ranters, The',
  'ended': True,
  'gid': 'c42eed94-e233-44e2-82b8-3ed6dd9bf318',
  'type': 'Group',
  'id': 153193},
 {'begin': {'year': 1956},
  'end': {'year': 1993},
  'name': 'The Silhouettes',
  'area': 'United States',
  'sort_name': 'Silhouettes, The',
  'ended': True,
  'gid': 'ca3f3ee1-c4a7-4bac-a16a-0

In [2]:
import redis

# redis サーバの設定
# ホスト名はDocke Composeで設定した名前を指定するとDocker側で名前解決してくれる
r = redis.StrictRedis(host='redis', port=6379, db=0)
pipe = r.pipeline()
pipe.flushall()
for i in artists:
    if 'area' in i.keys():
        # 名前の重複に対応するため名前とidをアンダーバーでつなげたものをキーとした
        pipe.set(i['name'] + "_" + str(i['id']), i['area'])

pipe.keys()
pipe.execute()[-1][:10]

[b'Jos\xc3\xa9 Manuel Silva Rodr\xc3\xadguez_1029292',
 b'Val McDermid_409980',
 b'angela_139328',
 b'Boa_849285',
 b'\xce\xa4\xce\xac\xcf\x83\xce\xbf\xcf\x82 \xce\x9c\xce\xb1\xce\xba\xcf\x81\xce\xbf\xce\xb3\xce\xb9\xce\xb1\xce\xbd\xce\xbd\xcf\x8c\xcf\x80\xce\xbf\xcf\x85\xce\xbb\xce\xbf\xcf\x82_1008911',
 b'The Swills_930027',
 b'Frozen Soul_1126313',
 b'NoRMAhl_35156',
 b'Undeclinable_89424',
 b'D.C.3_264549']

### 61. KVSの検索

60で構築したデータベースを用い，特定の（指定された）アーティストの活動場所を取得せよ．

In [3]:
for i in r.keys(pattern='Oasis_*'):
    print(i.decode('utf8') + ": " + r.get(i).decode('utf8'))

Oasis_286198: United States
Oasis_20660: United Kingdom
Oasis_377879: United Kingdom


### 62. KVS内の反復処理

60で構築したデータベースを用い，活動場所が「Japan」となっているアーティスト数を求めよ．

In [4]:
len([i for i in r.keys() if r.get(i).decode('utf8') == "Japan"])

22821

### 63. オブジェクトを値に格納したKVS

KVSを用い，アーティスト名（name）からタグと被タグ数（タグ付けされた回数）のリストを検索するためのデータベースを構築せよ．さらに，ここで構築したデータベースを用い，アーティスト名からタグと被タグ数を検索せよ．

In [5]:
r_tags = redis.StrictRedis(host='redis', port=6379, db=1)
tags_pipe = r_tags.pipeline()
tags_pipe.flushall()
for i in artists:
    if 'tags' in i.keys():
        for j in i['tags']:
            tags_pipe.lpush(i['name'] + "_" + str(i['id']), json.dumps(j))

tags_pipe.keys()
tags_pipe.execute()[-1][:10]

[b'Dear Winesburg_649270',
 b'Fist of the North Star_789002',
 b'Harry Edwards_326051',
 b'P\xc4\xb1nar Eli\xc3\xa7e_541543',
 b'Moshe Atzmon_818987',
 b'Cliff Martinez_48233',
 b'George Trevare_659671',
 b'Jonathan Pierre_709459',
 b'Joe B.G._823652',
 b'\xeb\xb0\x94\xed\x8a\xb8_517065']

In [6]:
# タグ数と被タグ数
for i in r_tags.lrange('Bouwe Dykstra_707059', 0, -1):
    tag = json.loads(i.decode('utf8').replace("'", '"'))
    print("%s: %d" % (tag['value'], tag['count']))

choral: 1
production music: 1


### 64. MongoDBの構築

アーティスト情報（artist.json.gz）をデータベースに登録せよ．さらに，次のフィールドでインデックスを作成せよ: name, aliases.name, tags.value, rating.value

In [7]:
import pymongo

# mongodb へのアクセスを確立
mongo = pymongo.MongoClient('mongo', 27017)
# DBの作成
db = mongo.nlp100
db.artists.drop()
# コレクションの作成
collection = db.artists

コレクションに `artists` のオブジェクトをinsert

In [8]:
# バルクインサートするぞい〜
collection.insert_many(artists)

### 65. MongoDBの検索

MongoDBのインタラクティブシェルを用いて，"Queen"というアーティストに関する情報を取得せよ．さらに，これと同様の処理を行うプログラムを実装せよ．

In [9]:
# インタラクティブシェル使えって書いてあるけど、どうやったらnotebook内でインタラクティブシェル使えるのかわからないので省略
list(collection.find({"name": "Queen"}))

[{'_id': ObjectId('5ffc14217f0306b84f92065d'),
  'name': 'Queen',
  'area': 'Japan',
  'gender': 'Female',
  'tags': [{'count': 1, 'value': 'kamen rider w'},
   {'count': 1, 'value': 'related-akb48'}],
  'sort_name': 'Queen',
  'ended': True,
  'gid': '420ca290-76c5-41af-999e-564d7c71f1a7',
  'type': 'Character',
  'id': 701492,
  'aliases': [{'name': 'Queen', 'sort_name': 'Queen'}]},
 {'_id': ObjectId('5ffc14227f0306b84f92cd09'),
  'rating': {'count': 24, 'value': 92},
  'begin': {'date': 27, 'month': 6, 'year': 1970},
  'name': 'Queen',
  'area': 'United Kingdom',
  'tags': [{'count': 2, 'value': 'hard rock'},
   {'count': 1, 'value': '70s'},
   {'count': 1, 'value': 'queen family'},
   {'count': 1, 'value': '90s'},
   {'count': 1, 'value': '80s'},
   {'count': 1, 'value': 'glam rock'},
   {'count': 4, 'value': 'british'},
   {'count': 1, 'value': 'english'},
   {'count': 2, 'value': 'uk'},
   {'count': 1, 'value': 'pop/rock'},
   {'count': 1, 'value': 'pop-rock'},
   {'count': 1, 'v

### 66. 検索件数の取得

MongoDBのインタラクティブシェルを用いて，活動場所が「Japan」となっているアーティスト数を求めよ

In [10]:
collection.count_documents({"area": "Japan"})

22821

### 67. 複数のドキュメントの取得

特定の（指定した）別名を持つアーティストを検索せよ．

In [11]:
# aliasesに別名が入ってる
list(collection.find({"aliases.name": "Silhouettes"}))

[{'_id': ObjectId('5ffc14127f0306b84f88cf3e'),
  'begin': {'year': 1956},
  'end': {'year': 1993},
  'name': 'The Silhouettes',
  'area': 'United States',
  'sort_name': 'Silhouettes, The',
  'ended': True,
  'gid': 'ca3f3ee1-c4a7-4bac-a16a-0b888a396c6b',
  'type': 'Group',
  'id': 101060,
  'aliases': [{'name': 'Silhouettes', 'sort_name': 'Silhouettes'},
   {'name': 'The Sihouettes', 'sort_name': 'The Sihouettes'}]}]

### 68. ソート

"dance"というタグを付与されたアーティストの中でレーティングの投票数が多いアーティスト・トップ10を求めよ．

In [12]:
# tags.valueがdanceのもので絞り込み
dance_collection = collection.find({"tags.value": "dance"})

# rating.countを降順でソートし最初の10個を取ってくる
list(dance_collection.sort("rating.count", pymongo.DESCENDING).limit(10))

[{'_id': ObjectId('5ffc14247f0306b84f93e05a'),
  'rating': {'count': 26, 'value': 88},
  'begin': {'date': 16, 'month': 8, 'year': 1958},
  'name': 'Madonna',
  'area': 'United States',
  'gender': 'Female',
  'tags': [{'count': 1, 'value': 'dance-pop'},
   {'count': 1, 'value': 'electropop'},
   {'count': 1, 'value': 'tell me'},
   {'count': 1, 'value': 'pop and chart'},
   {'count': 1, 'value': 'multiple ipi'},
   {'count': 1, 'value': 'electronic'},
   {'count': 1, 'value': 'américain'},
   {'count': 1, 'value': 'usa'},
   {'count': 1, 'value': 'singer'},
   {'count': 1, 'value': 'chanteur'},
   {'count': 1, 'value': 'american'},
   {'count': 4, 'value': 'pop'},
   {'count': 1, 'value': 'greatest hits'},
   {'count': 1, 'value': 'dance'}],
  'sort_name': 'Madonna',
  'ended': True,
  'gid': '79239441-bfd5-4981-a70c-55c3f15c1287',
  'type': 'Person',
  'id': 89},
 {'_id': ObjectId('5ffc14237f0306b84f9309e4'),
  'rating': {'count': 23, 'value': 84},
  'begin': {'date': 21, 'month': 11

### 69. Webアプリケーションの作成

ユーザから入力された検索条件に合致するアーティストの情報を表示するWebアプリケーションを作成せよ．アーティスト名，アーティストの別名，タグ等で検索条件を指定し，アーティスト情報のリストをレーティングの高い順などで整列して表示せよ．

_省略!!!!!!!!_